In [27]:
import json
import pandas as pd

In [28]:
geojson_path = 'vancouver.json'
van_json = json.loads(open(geojson_path).read())

In [29]:
num_nei = set()
for a in van_json['features']:
    num_nei.add(a['properties']['name'])

In [30]:
data_path = '../data/crimedata_csv_all_years.csv'
van_pd = pd.read_csv(data_path)

In [31]:
van_pd_nei = set(van_pd['NEIGHBOURHOOD'].unique())

In [32]:
num_nei - van_pd_nei

{'Downtown', 'Downtown Eastside', 'Dunbar Southlands', 'Strathcona\n'}

In [33]:
van_pd_nei - num_nei

{'Central Business District',
 'Dunbar-Southlands',
 'Musqueam',
 'Stanley Park',
 'Strathcona',
 nan}